In [ ]:
# what steps are happening here

# let's get a list of distinct HK device ID's and names 

# then let's call a function to parse all HR's for each device
# then save that DF

# in the meantime am I alright with my sensors being aggregates if that's what I'm into
# hell yeee 
# but for apple there are two watches that are getting samples for the same time
# I'd like to decide how to combine them, NGL also with priority, and later with kalman filters for smoothing


# a reminder the only reason for seperating at this point is the concurrent data points
# so yes, we'll be parsing the export twice

In [1]:
import xml.etree.ElementTree as ET 

exportLocation = "/home/chowder/Documents/dataExports/apple/2025-1-15/export/apple_health_export/export.xml"


In [20]:

# the function we'll make will take in the export file and return a list of source names , versions, and device metrics like ID's for a given record type
def getHKDeviceInfo(targetRecordType, record):
    # check if the type of record is right
    if (record["type"] == targetRecordType):

        # parse the device to a dictonary to make sure we have the right one
        # raw text of record["device"] should look like this 
        # '<<HKDevice: 0x999999999>, name:Apple Watch, manufacturer:' +
        # 'Apple Inc., model:Watch, hardware:Watch6,1, software:7.6>'
        device = [[x.split(":")[0].strip(), x.split(":")[1]] 
                  for x in record["device"].split(",")
                  if len(x.split(":")) == 2]

        device = [x for sublist in device for x in sublist]

        device[0] = device[0][3:]
        device[1] = device[1][:-1]
        # device[-1] = device[-1][:-1] # removes a hanging '>' from the field

        return device
    
    return None


def getDevicesForRecordType(targetRecordType, exportLocation):
    numRecords = 0
    totalRecords = 0
    devices = set()

    for event, elem in ET.iterparse(exportLocation, events=("start", "end")):
        totalRecords += 1
        # if totalRecords % 1_000_000 == 0: print(f"{totalRecords}")
        if event == "start" and elem.tag == "Record":
            record = elem.attrib
            deviceInfo = getHKDeviceInfo(targetRecordType, record)
            if deviceInfo is not None:
                numRecords += 1
                if numRecords % 1_000_000 == 0: print(f"cuurently found {numRecords}")

                sourceData = ["sourceName", record["sourceName"], "sourceVersion", record["sourceVersion"]]
                sourceData += deviceInfo
                devices.add(tuple(sourceData[1::2]))
        elem.clear() # this is important
    
    print(f"parsed total {totalRecords}")
    print(f"found {numRecords} relevant records")
    return devices

In [ ]:
sourceInfoNames = ["sourceName", "sourceVersion", "HKDeviceID", "deviceName", "manufacturer", "model", "hardwareVersion", "softwareVersion"]

d = getDevices("HKQuantityTypeIdentifierHeartRate", exportLocation)

cuurently found 1000000
cuurently found 2000000
parsed total 31764048
found 2645153 relevant records


In [15]:
list(set([x[0] for x in list(d)]))

['Abhik_AppleWatch_10_46mm_0',
 'Abhik’s Apple\xa0Watch',
 'Abhik_AppleWatch_6_40mm_0']

In [17]:
len(list(set([x[2] for x in list(d)])))

492

In [ ]:
#TODO change the underscores in the name to dashes
#make a function to go through the HR's and grab the values for just the device Name I'm looking for
#then write that to a parquet

#do the same with the other watch

In [22]:
from datetime import datetime
datetime.fromisoformat("2021-07-19 19:55:11 -0700")

datetime.datetime(2021, 7, 19, 19, 55, 11, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))

In [30]:
from datetime import datetime
import pandas as pd

def getAppleHKSamplesForDevice(targetSourceName, targetRecordType, exportLocation):
    # samples are of the format "SampleDT", "Value"
    numRecords = 0
    totalRecords = 0
    data = []

    for event, elem in ET.iterparse(exportLocation, events=("start", "end")):
        totalRecords += 1
        # if totalRecords % 1_000_000 == 0: print(f"{totalRecords}")

        if event != "start" or elem.tag != "Record":
            elem.clear()
            continue
        record = elem.attrib
        if record["type"] != targetRecordType or record["sourceName"] != targetSourceName:
            elem.clear()
            continue

        numRecords += 1
        if numRecords % 1_000_000 == 0: print(f"cuurently found {numRecords}")
        newRow = {"sampleDT": datetime.fromisoformat(record["startDate"]), "value": float(record["value"])}
        data.append(newRow)
        elem.clear() # this is important
    
    print(f"parsed total {totalRecords}")
    print(f"found {numRecords} relevant records")
    df = pd.DataFrame(data)
    df = df.set_index("sampleDT")
    return df

In [31]:
watch10Data = getAppleHKSamplesForDevice('Abhik_AppleWatch_10_46mm_0', "HKQuantityTypeIdentifierHeartRate", exportLocation)
watch10Data

cuurently found 1000000
parsed total 31764048
found 1503600 relevant records


In [ ]:
watch10Data

In [47]:
import importlib
importlib.reload(writeWorkingTSDf)

<module 'writeWorkingTSDf' from '/home/chowder/Documents/dataImport/writeWorkingTSDf.py'>

In [48]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
import writeWorkingTSDf
from writeWorkingTSDf import writeWorkingTSDf

In [49]:
writeWorkingTSDf("apple", "watch10", "hr", "builtin", watch10Data)

1503600 is too many rows writing 0 to 381107
saved to a file named 2024-09-28T100033+0000_2024-10-26T050151+0000_99a0370d_.parquet.gzip
1122493 is too many rows writing 381107 to 762214
saved to a file named 2024-10-26T050151+0000_2024-11-21T070208+0000_b6e6cedb_.parquet.gzip
saved to a file named 2024-11-21T070208+0000_2025-01-14T204308+0000_d46165ac_.parquet.gzip


In [50]:
watch6Data = getAppleHKSamplesForDevice('Abhik_AppleWatch_6_40mm_0', "HKQuantityTypeIdentifierHeartRate", exportLocation)
watch6Data

cuurently found 1000000
parsed total 31764048
found 1067153 relevant records


,value
sampleDT,
2022-05-09 20:02:34-07:00,84.3332
2022-05-09 20:10:28-07:00,105.0000
2022-05-09 20:05:48-07:00,79.0000
2022-05-09 20:11:39-07:00,71.0000
2022-05-09 20:16:16-07:00,66.0000
...,...
2024-09-11 09:08:41-07:00,64.9493
2024-09-11 09:18:36-07:00,71.1941
2024-09-11 09:28:41-07:00,76.8127


In [51]:
writeWorkingTSDf("apple", "watch6", "hr", "builtin", watch10Data)

1503600 is too many rows writing 0 to 381107
saved to a file named 2024-09-28T100033+0000_2024-10-26T050151+0000_99a0370d_.parquet.gzip
1122493 is too many rows writing 381107 to 762214
saved to a file named 2024-10-26T050151+0000_2024-11-21T070208+0000_b6e6cedb_.parquet.gzip
saved to a file named 2024-11-21T070208+0000_2025-01-14T204308+0000_d46165ac_.parquet.gzip


In [52]:
weirdWatchData = getAppleHKSamplesForDevice('Abhik’s Apple\xa0Watch', "HKQuantityTypeIdentifierHeartRate", exportLocation)
weirdWatchData

parsed total 31764048
found 74400 relevant records


,value
sampleDT,
2021-07-19 19:55:06-07:00,78.0000
2021-07-19 19:55:11-07:00,79.0000
2021-07-19 19:55:14-07:00,80.0000
2021-07-19 19:55:21-07:00,82.0000
2021-07-19 19:55:23-07:00,83.0000
...,...
2022-05-09 18:22:08-07:00,70.0000
2022-05-09 18:26:54-07:00,76.0000
2021-12-20 13:32:15-07:00,93.0261


In [56]:
writeWorkingTSDf("apple", "unknownWatch", "hr", "builtin", weirdWatchData)

saved to a file named 2021-07-20T025506+0000_2022-04-29T191017+0000_46b4750f_.parquet.gzip


In [53]:
d

{('Abhik_AppleWatch_6_40mm_0',
  '11.1',
  ' 0x3005c40f0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '11.1'),
 ('Abhik’s Apple\xa0Watch',
  '8.4.2',
  ' 0x300596ee0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '8.4.2'),
 ('Abhik_AppleWatch_6_40mm_0',
  '9.1',
  ' 0x3005977a0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '9.1'),
 ('Abhik_AppleWatch_6_40mm_0',
  '10.6.1',
  ' 0x3005f9860',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '10.6.1'),
 ('Abhik_AppleWatch_6_40mm_0',
  '10.6',
  ' 0x300630140',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '10.6'),
 ('Abhik_AppleWatch_6_40mm_0',
  '10.6.1',
  ' 0x30056c6e0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '10.6.1'),
 ('Abhik_AppleWatch_6_40mm_0',
  '9.6.3',
  ' 0x300556a30',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '9.6.3'),
 ('Abhik_AppleWatch_10_46mm_0',
  '11.2',
  ' 0x3005aba70',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watc

In [55]:
sorted([x for x in list(d) if x[0] == "Abhik’s Apple\xa0Watch"])

[('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x30056c870',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x30056ead0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x30057a0d0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x3005878e0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x3005932a0',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x300593660',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x300593890',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7.6',
  ' 0x3005d0d20',
  'Apple Watch',
  'Apple Inc.',
  'Watch',
  'Watch6',
  '7.6'),
 ('Abhik’s Apple\xa0Watch',
  '7